In [1]:
#!pip install datasets
#!pip install --upgrade --force-reinstall huggingface_hub

In [2]:
import torch
import torchvision
from torchvision.transforms import transforms, Lambda, Resize
from torchvision import datasets, transforms, models
import os
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Specify where to find the data preparation class
import sys
sys.path.append('../../Data_Preparation')
from Preparation import CustomDataLoader

In [3]:
# ResNet50 training data (ImageNet) properties
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
#DIMENSIONS = 3
#SIZE = 256

LR = 0.01
MOMENTUM=0.9
WEIGHT_DECAY = 1e-4
LR_STEP_SIZE = 5
LR_GAMMA = 0.1

In [4]:
dataloaders = {x: CustomDataLoader(data_path="../../FER2013_Data", batch_size=16, dataset_type=x, mean=MEAN, std=STD, dimensions=3).data_loader for x in ['train', 'test']}
dataset_sizes = {x: len(dataloaders[x]) for x in ['train', 'test']}
print(dataset_sizes)

class_names = dataloaders['train'].dataset.classes
print(class_names)

# Confirm correct data load
print("Train Data Loader:")
for batch_idx, (inputs, labels) in enumerate(dataloaders['train']):
    print("Batch Index:", batch_idx)
    print("Inputs Shape:", inputs.shape)
    print("Labels Shape:", labels.shape)
    # Print the first few labels in the batch
    print("Labels:", labels[:5])
    # Break after printing a few batches
    if batch_idx == 2:
        break

{'train': 1753, 'test': 449}
['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Train Data Loader:
Batch Index: 0
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([3, 3, 3, 2, 4])
Batch Index: 1
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([6, 3, 5, 3, 5])
Batch Index: 2
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([4, 3, 0, 6, 3])


In [5]:
dataloaders['test']

In [6]:
# Load the pre-trained ResNet-50 model
model = models.resnet50(pretrained=True)

# Freeze all layers except the final classification layer
for name, param in model.named_parameters():
    if "fc" in name:  # Unfreeze the final classification layer
        param.requires_grad = True
    else:
        param.requires_grad = False

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(
    model.parameters(),
    lr=LR,
    momentum=MOMENTUM,
    weight_decay=WEIGHT_DECAY
)

LR_SCHEDULER = torch.optim.lr_scheduler.StepLR(optimizer, step_size=LR_STEP_SIZE, gamma=LR_GAMMA)


# Move the model to the GPU if available
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = model.to(device)

C:\Users\mk6812ct\Downloads\Facial_Emotion_Detec-main\Facial_Emotion_Detec-main\.env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\mk6812ct\Downloads\Facial_Emotion_Detec-main\Facial_Emotion_Detec-main\.env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# num_epochs = 20
# for epoch in tqdm(range(num_epochs)):
#     for phase in ['train', 'test']:
#         if phase == 'train':
#             model.train()
#         else:
#             model.eval()

#         running_loss = 0.0
#         running_corrects = 0

#         for inputs, labels in dataloaders[phase]:
#             inputs = inputs.to(device)
#             labels = labels.to(device)

#             if phase == 'train':

#                 optimizer.zero_grad()

#                 with torch.set_grad_enabled(phase == 'train'):
#                     outputs = model(inputs)
#                     _, preds = torch.max(outputs, 1)
#                     loss = criterion(outputs, labels)

#                     if phase == 'train':
#                         loss.backward()
#                         optimizer.step()

#                 running_loss += loss.item() * inputs.size(0)
#                 running_corrects += torch.sum(preds == labels.data)

#             LR_SCHEDULER.step()

#             elif phase == 'test':
#                 with torch.inference_mode():  
#                     running_loss = 0
#                     for step, (images, labels) in enumerate(dataloaders[phase]):
#                         images, labels = images.to(device), labels.to(device)
                        
#                         outputs = model(images)
#                         loss = criterion(outputs, labels)

#                         running_loss += loss.item()

#     epoch_loss = running_loss / dataset_sizes[phase]
#     epoch_acc = running_corrects.double() / dataset_sizes[phase]

#     print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
# print("Training complete!")

In [8]:
num_epochs = 20

print("[INFO] Start training")
print("---------------------")

for epoch in tqdm(range(num_epochs)):

    total_batch = len(dataloaders['train'].dataset)//16
    
    # Training phase
    model.train()  # Set the model to train mode

    # Get statistics
    epoch_loss = 0
    len_dataset = 0
    
    for step, (batch_images, batch_labels) in enumerate(dataloaders['train']):
        X, Y = batch_images.to(device), batch_labels.to(device)

        pred = model(X)
        loss = criterion(pred, Y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += pred.shape[0] * loss.item()
        len_dataset += pred.shape[0]
        if (step) % 10 == 0:
            print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                 %(epoch+1, num_epochs, step+1, total_batch, loss.item()))

    epoch_loss = epoch_loss/ len_dataset
    print('Epoch: ', epoch+1, '| train loss : %0.4f' % epoch_loss)

    LR_SCHEDULER.step()
    
    # Validation phase
    model.eval()  # Set the model to evaluation mode
    with torch.inference_mode():  
        running_loss = 0
        for step, (images, labels) in enumerate(dataloaders['test']):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            
    running_loss = running_loss / len(dataloaders['test'])
    print('Epoch: ', epoch, '| test loss : %0.4f' % running_loss )

[INFO] Start training
---------------------


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch [1/20], lter [1/1752] Loss: 8.5280
Epoch [1/20], lter [11/1752] Loss: 3.5170
Epoch [1/20], lter [21/1752] Loss: 3.0042
Epoch [1/20], lter [31/1752] Loss: 3.0312
Epoch [1/20], lter [41/1752] Loss: 3.0662
Epoch [1/20], lter [51/1752] Loss: 2.0513
Epoch [1/20], lter [61/1752] Loss: 3.3191
Epoch [1/20], lter [71/1752] Loss: 1.6901
Epoch [1/20], lter [81/1752] Loss: 2.7134
Epoch [1/20], lter [91/1752] Loss: 2.0342
Epoch [1/20], lter [101/1752] Loss: 1.5432
Epoch [1/20], lter [111/1752] Loss: 2.2246
Epoch [1/20], lter [121/1752] Loss: 1.9009
Epoch [1/20], lter [131/1752] Loss: 2.3794
Epoch [1/20], lter [141/1752] Loss: 4.2546
Epoch [1/20], lter [151/1752] Loss: 1.4273
Epoch [1/20], lter [161/1752] Loss: 1.3442
Epoch [1/20], lter [171/1752] Loss: 2.9274
Epoch [1/20], lter [181/1752] Loss: 2.4359
Epoch [1/20], lter [191/1752] Loss: 1.7520
Epoch [1/20], lter [201/1752] Loss: 4.1641
Epoch [1/20], lter [211/1752] Loss: 4.7436
Epoch [1/20], lter [221/1752] Loss: 1.8984
Epoch [1/20], lter [23

  5%|▌         | 1/20 [27:39<8:45:35, 1659.77s/it]

Epoch:  0 | test loss : 1.7332
Epoch [2/20], lter [1/1752] Loss: 1.4411
Epoch [2/20], lter [11/1752] Loss: 1.4073
Epoch [2/20], lter [21/1752] Loss: 2.3826
Epoch [2/20], lter [31/1752] Loss: 2.0099
Epoch [2/20], lter [41/1752] Loss: 2.8085
Epoch [2/20], lter [51/1752] Loss: 1.7508
Epoch [2/20], lter [61/1752] Loss: 2.7304
Epoch [2/20], lter [71/1752] Loss: 3.4405
Epoch [2/20], lter [81/1752] Loss: 2.4046
Epoch [2/20], lter [91/1752] Loss: 2.3934
Epoch [2/20], lter [101/1752] Loss: 2.1200
Epoch [2/20], lter [111/1752] Loss: 1.8039
Epoch [2/20], lter [121/1752] Loss: 2.6497
Epoch [2/20], lter [131/1752] Loss: 1.8135
Epoch [2/20], lter [141/1752] Loss: 1.7242
Epoch [2/20], lter [151/1752] Loss: 2.4385
Epoch [2/20], lter [161/1752] Loss: 3.4280
Epoch [2/20], lter [171/1752] Loss: 2.3567
Epoch [2/20], lter [181/1752] Loss: 6.4756
Epoch [2/20], lter [191/1752] Loss: 3.2092
Epoch [2/20], lter [201/1752] Loss: 4.0802
Epoch [2/20], lter [211/1752] Loss: 1.6955
Epoch [2/20], lter [221/1752] Loss

...

Epoch:  17 | test loss : 1.4314
Epoch [19/20], lter [1/1752] Loss: 1.4007
Epoch [19/20], lter [11/1752] Loss: 1.3602
Epoch [19/20], lter [21/1752] Loss: 1.2583
Epoch [19/20], lter [31/1752] Loss: 1.2139
Epoch [19/20], lter [41/1752] Loss: 1.5530
Epoch [19/20], lter [51/1752] Loss: 1.4511
Epoch [19/20], lter [61/1752] Loss: 1.5383
Epoch [19/20], lter [71/1752] Loss: 1.3878
Epoch [19/20], lter [81/1752] Loss: 1.5036
Epoch [19/20], lter [91/1752] Loss: 1.6198
Epoch [19/20], lter [101/1752] Loss: 1.5806
Epoch [19/20], lter [111/1752] Loss: 0.9200
Epoch [19/20], lter [121/1752] Loss: 1.3399
Epoch [19/20], lter [131/1752] Loss: 1.6258
Epoch [19/20], lter [141/1752] Loss: 1.3706
Epoch [19/20], lter [151/1752] Loss: 1.7625
Epoch [19/20], lter [161/1752] Loss: 1.7452
Epoch [19/20], lter [171/1752] Loss: 1.3867
Epoch [19/20], lter [181/1752] Loss: 1.3914
Epoch [19/20], lter [191/1752] Loss: 1.5240
Epoch [19/20], lter [201/1752] Loss: 1.6064
Epoch [19/20], lter [211/1752] Loss: 1.5161
Epoch [19/2

 95%|█████████▌| 19/20 [8:43:17<27:31, 1651.61s/it]

Epoch:  18 | test loss : 1.4195
Epoch [20/20], lter [1/1752] Loss: 1.5243
Epoch [20/20], lter [11/1752] Loss: 1.3079
Epoch [20/20], lter [21/1752] Loss: 1.5430
Epoch [20/20], lter [31/1752] Loss: 1.0697
Epoch [20/20], lter [41/1752] Loss: 0.9020
Epoch [20/20], lter [51/1752] Loss: 1.8747
Epoch [20/20], lter [61/1752] Loss: 1.5185
Epoch [20/20], lter [71/1752] Loss: 1.0407
Epoch [20/20], lter [81/1752] Loss: 1.4169
Epoch [20/20], lter [91/1752] Loss: 1.4658
Epoch [20/20], lter [101/1752] Loss: 1.3180
Epoch [20/20], lter [111/1752] Loss: 1.1695
Epoch [20/20], lter [121/1752] Loss: 0.8459
Epoch [20/20], lter [131/1752] Loss: 1.4879
Epoch [20/20], lter [141/1752] Loss: 1.4309
Epoch [20/20], lter [151/1752] Loss: 2.0136
Epoch [20/20], lter [161/1752] Loss: 1.4160
Epoch [20/20], lter [171/1752] Loss: 1.1937
Epoch [20/20], lter [181/1752] Loss: 1.1949
Epoch [20/20], lter [191/1752] Loss: 1.5975
Epoch [20/20], lter [201/1752] Loss: 1.9980
Epoch [20/20], lter [211/1752] Loss: 1.1770
Epoch [20/2

100%|██████████| 20/20 [9:10:54<00:00, 1652.74s/it]

Epoch:  19 | test loss : 1.4339


In [9]:
correct_predictions, total_predictions = 0, 0

# Loop over the batches of the test set
for batch_data in dataloaders['train']:
    # Forward pass
    inputs, target = batch_data
    output = model(inputs)

    # Get the predicted class
    _, predicted = torch.max(output.data, 1)

    # Increase the total predictions counter
    total_predictions += target.size(0)

    # Check if the predicted class is equal to the true class
    correct_predictions += (predicted == target).sum().item()

# Calculate the accuracy
accuracy = correct_predictions / total_predictions
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 47.68%


In [10]:
correct_predictions, total_predictions = 0, 0

# Loop over the batches of the test set
for batch_data in dataloaders['test']:
    # Forward pass
    inputs, target = batch_data
    output = model(inputs)

    # Get the predicted class
    _, predicted = torch.max(output.data, 1)

    # Increase the total predictions counter
    total_predictions += target.size(0)

    # Check if the predicted class is equal to the true class
    correct_predictions += (predicted == target).sum().item()

# Calculate the accuracy
accuracy = correct_predictions / total_predictions
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 44.11%


In [11]:
# Save the model
torch.save(model.state_dict(), 'ResNet50.pth')